# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from pytrends.request import TrendReq

g = globals()

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
# 검색어 수집 반복문
import collect_keywords
import asyncio
async def main():
    # collect_keywords 함수를 호출하고 결과를 기다립니다.
    tmp_df = await collect_keywords.collect_keywords(srch_keyword, day)
    print(tmp_df)
    return tmp_df
# 이미 실행 중인 이벤트 루프를 얻습니다.


tmp_df = await main()
print(tmp_df)



# import collect_keywords
# start=time.time()
# tmp_df =  collect_keywords.asyncio.run(main())
# end=time.time()

keyword_final
keyword_final 연관검색어 추출 시작!!
33% complete!
66% complete!
100% complete!
     연관키워드   월간검색수_합계
0   삼성전자주가  4520800.0
1     달러환율  2061000.0
2      나스닥   784700.0
3       주식   528000.0
4   검색상위종목   516940.0
..     ...        ...
20  연금저축펀드    45900.0
21    청년대출    45500.0
22    개인연금    45400.0
23    소액대출    45000.0
24    이자계산    44900.0

[150 rows x 2 columns]
     연관키워드   월간검색수_합계
0   삼성전자주가  4520800.0
1     달러환율  2061000.0
2      나스닥   784700.0
3       주식   528000.0
4   검색상위종목   516940.0
..     ...        ...
20  연금저축펀드    45900.0
21    청년대출    45500.0
22    개인연금    45400.0
23    소액대출    45000.0
24    이자계산    44900.0

[150 rows x 2 columns]


# 4. 키워드 트렌드 및 패턴 분석

1) 정보지정

In [7]:
import utils
    # 날짜 지정
today = datetime.now(timezone('Asia/Seoul'))
day = today.strftime("%y%m%d")  

    # 저장할 폴더 미리 생성, 폴더가 이미 있으면 생성하지 않음
folders_to_create = [
        './data/tmp',
        './data/result',
        f'./data/result/{day}',
        f'./data/result/{day}/graph'
    ]

for folder in folders_to_create:
        os.makedirs(folder, exist_ok=True)

    # 분석 기간 설정
start_index = (today - relativedelta(days=1)).strftime("%Y-%m-%d")
end_index = (today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")
print(['분석시작일:', start_index, '분석종료일:', end_index])

    # URI 정보 가져오기
URI = utils.get_secret("URI")

['분석시작일:', '2024-03-05', '분석종료일:', '2021-03-05']


# 5. 상세 분석

In [8]:
# 삭제할 파일 경로 정의
progress_log_path = 'progress_log.txt'  # 가정: 실제 경로에 맞게 수정해야 함
progress_state_path = 'progress_state.json'  # 가정: 실제 경로에 맞게 수정해야 함

# progress_log 파일 삭제
if os.path.exists(progress_log_path):
    os.remove(progress_log_path)
    print(f'{progress_log_path} has been deleted.')
else:
    print(f'{progress_log_path} does not exist.')

# progress_state.json 파일 삭제
if os.path.exists(progress_state_path):
    os.remove(progress_state_path)
    print(f'{progress_state_path} has been deleted.')
else:
    print(f'{progress_state_path} does not exist.')

progress_log.txt has been deleted.
progress_state.json does not exist.


In [9]:
from analysis_test import analysis
import select_keyword

import nest_asyncio


nest_asyncio.apply()

start=time.time()
analysis_instance = analysis()
trend_data=analysis_instance.collect_and_analyze_keyword_trends()
analysis_instance.select_keywords(trend_data)
print(time.time()-start)

################################################ 주식 (1/3) ################################################
################################################ 금리 (2/3) ################################################
Current Client ID: 3PT4BXkVGd7R67Y5hhsE (100/1000) , index 49 of 842

################################################ 금융상품 (3/3) ################################################
API 요청 데이터 수: 150
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
월별 급상승 키워드 발견 : ISA계좌
          기준일자        유형  연관검색어       검색일자  검색량
0   2024-03-06

In [10]:
analysis_instance.select_keywords(trend_data)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
월별 급상승 키워드 발견 : ISA계좌
          기준일자        유형  연관검색어       검색일자  검색량
0   2024-03-06  month급상승  ISA계좌 2021-06-29  NaN
1   2024-03-06  month급상승  ISA계좌 2021-07-27  NaN
2   2024-03-06  month급상승  ISA계좌 2021-08-24  NaN
3   2024-03-06  month급상승  ISA계좌 2021-09-21  NaN
4   2024-03-06  month급상승  ISA계좌 2021-10-19  NaN
5   2024-03-06  month급상승  ISA계좌 2021-11-16  NaN
6   2024-03-06  month급상승  ISA계좌 2021-12-14  NaN
7   2024-03-06  month급상승  ISA계좌 2022-01-11  NaN
8   2024-03-06  month급상승  ISA계좌 2022-02-0

In [13]:
review = analysis_instance.keyword_data

{'keyword_categories': {},
 'related_search_terms': {},
 'related_search_volume': {},
 'up_month': {}}

### 동기 : 49.392563343048096


In [10]:
data_list=['daily','weekly_up','weekly_stay','monthly_up','monthly_stay','monthly_rule']
daily_up_trend, weekly_up_trend, weekly_stay_trend, monthly_up_trend, monthly_stay_trend, monthly_rule_trend = [pd.DataFrame() for _ in range(6)]

In [11]:
daily_up_info, weekly_up_info, weekly_stay_info, monthly_up_info, monthly_stay_info, monthly_rule_info = [{} for _ in range(6)]

In [12]:
daily_up_trendc=analysis.map_columns_based_on_keys(review_types, 'daily')

weekly_up_trend=analysis.map_columns_based_on_keys(review_types, 'weekly_up')

weekly_stay_trend=analysis.map_columns_based_on_keys(review_types, 'weekly_stay')

monthly_up_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_up')

monthly_stay_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_stay')

monthly_rule_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_rule')
    

One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.


In [13]:
dict_list = ['keyword_categories', 'related_search_terms', \
             'related_search_volume', 'daily_up_info',\
                  'weekly_up_info', 'weekly_stay_info', 'monthly_up_info', \
                    'monthly_stay_info', 'monthly_rule_info', 'up_month']

trend_type = dict(zip(['daily_up_trend', 'weekly_up_trend', \
                       'weekly_stay_trend', 'monthly_up_trend', 'monthly_stay_trend', 'monthly_rule_trend'] , 
                        ['일별 급상승', '주별 급상승', '주별 지속상승', '월별 급상승', '월별 지속상승', '월별 규칙성']))



In [14]:
# 결과 저장을 위한 폴더 경로
folders = ['./data/dict', './data/types', './data/json']

try:
    # 폴더 생성
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
        else:
            print(f"Folder '{folder}' already exists.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Folder './data/dict' already exists.
Folder './data/types' already exists.
Folder './data/json' already exists.


In [15]:
import pickle

# 유형 분류 결과 저장
for k in dict_list:
    with open(f'./data/dict/{k}.pkl', 'wb') as f:
        pickle.dump(eval(f'{k}'), f)

for type_eng, type_kor in trend_type.items():
    evaluated_df = eval(type_eng)  # 변수의 평가된 결과를 저장
    if evaluated_df is not None:  # 평가된 결과가 None이 아니면 CSV로 저장
        print(f"{type_eng}에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.")
        evaluated_df.to_csv(f'./data/types/{type_eng}_result.csv', encoding='utf-8-sig')
    else:  # 평가된 결과가 None이면 오류 메시지 출력
        print(f"Error for type_eng '{type_eng}': 'NoneType' object has no attribute 'to_csv'")


daily_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
weekly_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
Error for type_eng 'weekly_stay_trend': 'NoneType' object has no attribute 'to_csv'
monthly_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
Error for type_eng 'monthly_stay_trend': 'NoneType' object has no attribute 'to_csv'
Error for type_eng 'monthly_rule_trend': 'NoneType' object has no attribute 'to_csv'


# 파일 정리 코드


make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

이걸로 생성된 파일 삭제하는 코드

In [3]:
from utils import remove_directory

# 특정 디렉토리 삭제    
remove_directory('./data')

디렉토리 './data'가 삭제되었습니다.


# 로그파일 삭제

In [13]:
import multiprocessing

# 사용 가능한 CPU 코어의 수를 확인
cpu_count = multiprocessing.cpu_count()
print(f"Available CPU cores: {cpu_count}")

Available CPU cores: 16


1. 세션이 끊겼을 때 어떻게해야 좋을지 판단 로그파일을 인터넷에 깔아서 관리할까? 아니면 딕셔너리로 보이게 해서 관리할까? 라는 생각
2. 너무 하드코딩되어있는 부분 존재 이런부분은 처리하기
3. missing데이터 같은 경우 딕셔너리로 관리해서 daily나 weekly일때 다르게 구분하기 편하게 하기
4. 로그파일로 따로 관리하는거 좋은가? 에 대한 생각해보기
5. 주석같은거 중요한거마다 하기
6. 

In [2]:

def main():
    '''
    apiCallCount :로그 카운트
    maxKeyword:main_keyword['keyword_final']의 길이
    currentRequestCount : 각 api계정당 요청횟수(1000개 미만으로 제한)
    keywordName   : 키워드 네임
    i: main_keyword['keyword_final']의 인덱스
    '''
    #################### 저장 장소 생성############
    check_list=[]
    # 데이터프레임 초기화
    daily_up_trend, weekly_up_trend, weekly_stay_trend, monthly_up_trend, monthly_stay_trend, monthly_rule_trend = [pd.DataFrame() for _ in range(6)]
    graph_table, graph_table_day, graph_table_week, graph_table_month = [pd.DataFrame() for _ in range(4)]
    graph_table_day.index = pd.date_range(start=start_index, end=end_index, freq='1d')
    graph_table_week.index = pd.date_range(start=start_index, end=end_index, freq='7d')
    graph_table_month.index = pd.date_range(start=start_index, end=end_index, freq='28d')
    
    # 정보 저장을 위한 딕셔너리 초기화
    keyword_categories , related_search_terms , related_search_volume = {}, {}, {}
    daily_up_info, weekly_up_info, weekly_stay_info, monthly_up_info, monthly_stay_info, monthly_rule_info = [{} for _ in range(6)]
    review_types = {
            'daily': {
                    'function': select_keyword.select_keyword,
                    'info_dict': daily_up_info,
                    'trend_df': daily_up_trend,
                    'graph_df': graph_table_day,
                    'time_period': 'daily'
                },
        'weekly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': weekly_up_info,
                    'trend_df': weekly_up_trend,
                    'graph_df': graph_table_week,
                    'time_period': 'weekly'
                },
        'weekly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': weekly_stay_info,
                    'trend_df': weekly_stay_trend,
                    'graph_df': graph_table_week,
                    'time_period': 'weekly'
                },
        'monthly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': monthly_up_info,
                    'trend_df': monthly_up_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                },
        'monthly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': monthly_stay_info,
                    'trend_df': monthly_stay_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                },
        'monthly_rule': {
                    'function': select_keyword.monthly_rule,
                    'info_dict': monthly_rule_info,
                    'trend_df': monthly_rule_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                }
            }


    check_list = [] # 키워드 저장용

    up_month = {} # 상승하는 달

    


    related_search_volume={}

    #################### 저장 장소 생성############
    formatted_today, day=utils.get_today_date()
    state = utils.load_state()
    apiCallCount = state.get('api_request_count', 1)
    current_client_index = state.get('current_client_index', 0)
    clients = utils.get_secret("clients")  # 클라이언트 정보 로드
    
    keywords = utils.load_keywords('main_keyword.json')['keyword_final']
    total_keywords = len(keywords)
    request_limit = 1000  # 각 클라이언트별 요청 한도
    i = state['keyword_index']
    data_list=[]
    all_keywords_list=[]
    standard_time = datetime.now()
    api_url = "https://openapi.naver.com/v1/datalab/search"
    api_request_data = pd.DataFrame(columns=['search_keyword', 'client_id', 'client_secret', 'api_url'])

    
    keyword_categories,related_search_volume,related_search_terms,up_month,review_types\
        =src.analysis.analysis.collect_and_analyze_keyword_trends(keywords,state,day,total_keywords,\
                                       clients,request_limit,api_url,standard_time,review_types,up_month,apiCallCount,current_client_index,i,api_request_data,\
                                        keyword_categories,related_search_volume,related_search_terms)

    return keyword_categories,related_search_volume,related_search_terms,up_month,review_types

if __name__ == '__main__':
        start = time.time()
        keyword_categories,related_search_volume,related_search_terms,up_month,review_types=main()
        end = time.time()
        print(f'총 {end-start}초 소요')

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import json
import utils
import os
import trend as trend
import select_keyword
import analysis as analysis
def prepare_and_execute_api_requests(keywordName, day, state, clients, apiCallCount, current_client_index, request_limit, api_url):
    """
    주어진 키워드에 대해 API 요청 데이터를 준비하고 실행합니다.
    """
    # CSV 파일 로드
    df_table = pd.read_csv(f'./data/rl_srch/{day}/{keywordName}.csv', encoding='euc-kr')
    total_keywords = len(df_table)
    print(f'################################################ {keywordName} ({state["keyword_index"]+1}/{total_keywords}) ################################################')
    maxKeyword = min(50, len(df_table))  # 최대 연관검색어 수

    api_request_data = pd.DataFrame()  # API 요청 데이터를 저장할 데이터프레임

    for currentRequestCount in range(state['currentRequestCount_index'], maxKeyword):
        request_data, apiCallCount, current_client_index = analysis.handle_api_call(
            keywordName, currentRequestCount, df_table, clients, apiCallCount,
            current_client_index, request_limit, api_url)

        if request_data is not None:
            # 데이터 프레임에 API 요청 데이터 추가
            api_request_data = pd.concat([api_request_data, pd.DataFrame([request_data])], ignore_index=True)
        else:
            # API 요청 한도 초과 시 처리
            break

    if current_client_index >= len(clients):
        print("모든 API 클라이언트의 요청 한도 초과")
    
    i = state['keyword_index'] + 1
    print(f'API 요청 데이터 수: {len(api_request_data)}')

    return api_request_data, apiCallCount, current_client_index, i
        

def main():
    '''
    apiCallCount :로그 카운트
    maxKeyword:main_keyword['keyword_final']의 길이
    currentRequestCount : 각 api계정당 요청횟수(1000개 미만으로 제한)
    keywordName   : 키워드 네임
    i: main_keyword['keyword_final']의 인덱스
    '''
    #################### 저장 장소 생성############
    check_list=[]
    # 데이터프레임 초기화
    daily_up_trend, weekly_up_trend, weekly_stay_trend, monthly_up_trend, monthly_stay_trend, monthly_rule_trend = [pd.DataFrame() for _ in range(6)]
    graph_table, graph_table_day, graph_table_week, graph_table_month = [pd.DataFrame() for _ in range(4)]
    graph_table_day.index = pd.date_range(start=start_index, end=end_index, freq='1d')
    graph_table_week.index = pd.date_range(start=start_index, end=end_index, freq='7d')
    graph_table_month.index = pd.date_range(start=start_index, end=end_index, freq='28d')
    
    # 정보 저장을 위한 딕셔너리 초기화
    keyword_categories , related_search_terms , related_search_volume = {}, {}, {}
    daily_up_info, weekly_up_info, weekly_stay_info, monthly_up_info, monthly_stay_info, monthly_rule_info = [{} for _ in range(6)]
    review_types = {
            'daily': {
                    'function': select_keyword.select_keyword,
                    'info_dict': daily_up_info,
                    'trend_df': daily_up_trend,
                    'graph_df': graph_table_day,
                    'time_period': 'daily'
                },
        'weekly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': weekly_up_info,
                    'trend_df': weekly_up_trend,
                    'graph_df': graph_table_week,
                    'time_period': 'weekly'
                },
        'weekly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': weekly_stay_info,
                    'trend_df': weekly_stay_trend,
                    'graph_df': graph_table_week,
                    'time_period': 'weekly'
                },
        'monthly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': monthly_up_info,
                    'trend_df': monthly_up_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                },
        'monthly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': monthly_stay_info,
                    'trend_df': monthly_stay_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                },
        'monthly_rule': {
                    'function': select_keyword.monthly_rule,
                    'info_dict': monthly_rule_info,
                    'trend_df': monthly_rule_trend,
                    'graph_df': graph_table_month,
                    'time_period': 'month'
                }
            }


    check_list = [] # 키워드 저장용

    up_month = {} # 상승하는 달

    


    related_search_volume={}

    #################### 저장 장소 생성############
    formatted_today, day=utils.get_today_date()
    state = utils.load_state()
    apiCallCount = state.get('api_request_count', 1)
    current_client_index = state.get('current_client_index', 0)
    clients = utils.get_secret("clients")  # 클라이언트 정보 로드
    
    keywords = utils.load_keywords('main_keyword.json')['keyword_final']
    total_keywords = len(keywords)
    request_limit = 1000  # 각 클라이언트별 요청 한도
    i = state['keyword_index']
    standard_time = datetime.now()
    api_url = "https://openapi.naver.com/v1/datalab/search"
    api_request_data = pd.DataFrame(columns=['search_keyword', 'client_id', 'client_secret', 'api_url'])

        ## 설명
    for keywordName   in keywords[state['keyword_index']:]:
        df_table = pd.read_csv('./data/rl_srch/' + str(day) + f'/{keywordName}.csv', encoding='euc-kr')
        print(f'################################################ {keywordName } ({i+1}/{total_keywords}) ################################################')
        maxKeyword = min(50, len(df_table))   # 최대 연관검색어 수 (50개가 안되면 DF_TABLE의 길이만큼)

        for currentRequestCount  in range(state['currentRequestCount_index'], maxKeyword):
            # 설명

            request_data, apiCallCount, current_client_index = analysis.handle_api_call(
                keywordName, currentRequestCount, df_table, clients, apiCallCount, 
                current_client_index, request_limit, api_url)

            if request_data is not None:
                # 데이터 프레임에 API 요청 데이터 추가
                new_row = pd.DataFrame([request_data])
                api_request_data = pd.concat([api_request_data, new_row], ignore_index=True)
            else:
                # API 요청 한도 초과 시 처리
                break

        if current_client_index >= len(clients):
            print("모든 API 클라이언트의 요청 한도 초과")
            break

        i += 1 
        print(f'API 요청 데이터 수: {len(api_request_data)}')
        unique_api_request_data  = api_request_data.drop_duplicates(subset=['search_keyword'])
        print(f'유일한 API 요청 데이터 수: {len(unique_api_request_data)}')
        for index, row in unique_api_request_data.iterrows():
            if row['keywordName'] != keywordName:
                continue

            
            trend_data = trend.trend_load(standard_time, row['search_keyword'], row['client_id'], row['client_secret'], row['api_url'])

            for review_key, settings in review_types.items():
                # 'monthly_rule' 유형의 처리를 먼저 확인
                if review_key == 'monthly_rule':
                    tmp, tmp_gph, tmp_info, rising_month = settings['function'](trend_data, day, settings['time_period'])
                    # row[search_keyword] : 해당 키워드의 연관키워드 리스트
                    if tmp is not None:
                        if str(keywordName) not in up_month:
                            up_month[str(keywordName)] = rising_month
                    continue
                
                # 나머지 유형의 일반적인 처리
                tmp, tmp_gph, tmp_info = settings['function'](trend_data, day, settings['time_period'])

                if tmp is None:
                    continue

                # 결과 처리 로직에 사용될 인자들을 하나의 딕셔너리로 묶기
                check_result_args = {
                    'tmp': tmp,
                    'tmp_gph': tmp_gph,
                    'tmp_info': tmp_info,
                    'trend_df': settings['trend_df'],
                    'graph_df': settings['graph_df'],
                    'keywordName': keywordName,
                    'keyword_categories': keyword_categories,
                    'related_search_volume': related_search_volume,
                    'df_table': df_table,
                    'info_dict': settings['info_dict'],
                    'related_search_terms': related_search_terms,
                    # 'currentKeywordData',: row['search_keyword']
                }
    
                # **kwargs를 사용하여 딕셔너리를 함수에 전달
                keyword_categories, related_search_volume, settings['info_dict'], related_search_terms, settings['trend_df'], settings['graph_df'] = analysis.check_result(**check_result_args)

    return keyword_categories, related_search_volume, related_search_terms, review_types,rising_month
                    


if __name__ == '__main__':
        start = time.time()
        keyword_categories, related_search_volume,related_search_terms,review_types,up_month=main()
        end = time.time()
        print(f'총 {end-start}초 소요')
        

In [ ]:
import select_keyword

import analysis_test as analysis_test
def main():
    '''
    apiCallCount :로그 카운트
    maxKeyword:main_keyword['keyword_final']의 길이
    currentRequestCount : 각 api계정당 요청횟수(1000개 미만으로 제한)
    keywordName   : 키워드 네임
    i: main_keyword['keyword_final']의 인덱스
    '''
    #################### 저장 장소 생성############
    # 데이터프레임 초기화
    trends_dataframes = {
    'daily_up': pd.DataFrame(),
    'weekly_up': pd.DataFrame(),
    'weekly_stay': pd.DataFrame(),
    'monthly_up': pd.DataFrame(),
    'monthly_stay': pd.DataFrame(),
    'monthly_rule': pd.DataFrame()
    }


    graph_tables = {
    'day': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='1d')),
    'week': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='7d')),
    'month': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='28d'))
    }

    # 정보 저장을 위한 딕셔너리 초기화
    keyword_data = {
        'keyword_categories': {},
        'related_search_terms': {},
        'related_search_volume': {},
        'up_month': {}
    }
    review_types = {
            'daily': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['daily_up'],
                    'graph_df': graph_tables['day'],
                    'time_period': 'daily'
                },
        'weekly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['weekly_up'],
                    'graph_df': graph_tables['week'],
                    'time_period': 'weekly'
                },
        'weekly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': {},
                    'trend_df': trends_dataframes['weekly_stay'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'weekly'
                },
        'monthly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_up'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                },
        'monthly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_stay'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                },
        'monthly_rule': {
                    'function': select_keyword.monthly_rule,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_rule'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                }
            }
    

    


    #################### 저장 장소 생성############
    formatted_today, day=utils.get_today_date()
    state = utils.load_state()
    apiCallCount = state.get('api_request_count', 1)
    current_client_index = state.get('current_client_index', 0)
    clients = utils.get_secret("clients")  # 클라이언트 정보 로드
    
    keywords = utils.load_keywords('main_keyword.json')['keyword_final']
    total_keywords = len(keywords)
    request_limit = 1000  # 각 클라이언트별 요청 한도
    i = state['keyword_index']
    standard_time = datetime.now()
    api_url = "https://openapi.naver.com/v1/datalab/search"
    api_request_data = pd.DataFrame(columns=['search_keyword', 'client_id', 'client_secret', 'api_url'])

    # 로직!!!
    analysis=analysis_test.analysis()
    trends_dataframes,graph_tables,review_types,keyword_data=


    # keyword_categories,related_search_volume,related_search_terms,up_month,review_types\
    #     =analysis.collect_and_analyze_keyword_trends(keywords,state,day,total_keywords,\
    #                                    clients,request_limit,api_url,standard_time,review_types,up_month,apiCallCount,current_client_index,i,api_request_data,\
    #                                     keyword_categories,related_search_volume,related_search_terms)

    return keyword_categories,related_search_volume,related_search_terms,up_month,review_types

if __name__ == '__main__':
        start = time.time()
        keyword_categories,related_search_volume,related_search_terms,up_month,review_types=main()
        end = time.time()
        print(f'총 {end-start}초 소요')

In [ ]:
import select_keyword
import analysis as analysis
def main():
    '''
    apiCallCount :로그 카운트
    maxKeyword:main_keyword['keyword_final']의 길이
    currentRequestCount : 각 api계정당 요청횟수(1000개 미만으로 제한)
    keywordName   : 키워드 네임
    i: main_keyword['keyword_final']의 인덱스
    '''
    #################### 저장 장소 생성############
    # 데이터프레임 초기화
    trends_dataframes = {
    'daily_up': pd.DataFrame(),
    'weekly_up': pd.DataFrame(),
    'weekly_stay': pd.DataFrame(),
    'monthly_up': pd.DataFrame(),
    'monthly_stay': pd.DataFrame(),
    'monthly_rule': pd.DataFrame()
    }


    graph_tables = {
    'day': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='1d')),
    'week': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='7d')),
    'month': pd.DataFrame(index=pd.date_range(start=start_index, end=end_index, freq='28d'))
    }

    # 정보 저장을 위한 딕셔너리 초기화
    keyword_data = {
        'keyword_categories': {},
        'related_search_terms': {},
        'related_search_volume': {},
        'up_month': {}
    }
    review_types = {
            'daily': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['daily_up'],
                    'graph_df': graph_tables['day'],
                    'time_period': 'daily'
                },
        'weekly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['weekly_up'],
                    'graph_df': graph_tables['week'],
                    'time_period': 'weekly'
                },
        'weekly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': {},
                    'trend_df': trends_dataframes['weekly_stay'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'weekly'
                },
        'monthly_up': {
                    'function': select_keyword.select_keyword,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_up'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                },
        'monthly_stay': {
                    'function': select_keyword.rising_keyword_analysis,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_stay'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                },
        'monthly_rule': {
                    'function': select_keyword.monthly_rule,
                    'info_dict': {},
                    'trend_df': trends_dataframes['monthly_rule'],
                    'graph_df': graph_tables['month'],
                    'time_period': 'month'
                }
            }
    


    # 로직!!!
    analysis=analysis.analysis()
    trends_dataframes,graph_tables,review_types,keyword_data=analysis.collect_and_analyze_keyword_trends()
    return trends_dataframes,graph_tables,review_types,keyword_data





if __name__ == '__main__':
        start = time.time()
        keyword_categories,related_search_volume,related_search_terms,up_month,review_types=main()
        end = time.time()
        print(f'총 {end-start}초 소요')